# Stage 1 AI training: Input channels

This stage focuses on the **magnetic components** used as input for the machine learning models. Different architectures and rotational frames will be used for every magnetic component option.

In [ ]:
# Import packages:
import MLQDM.MLmodel as ML_MLmodel
import MLQDM.timewindows as ML_twdw
import tensorflow as tf

# Check available GPU:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

## Load original data and general parameters
Load data from files (many segments) and store information in dataframes, one for each segment. The original data is in the **Laboratory rotational frame (RF1)**.

There are two possible sets of target labels, coming for the 'linear approximation' of 'physical model' approaches regarding the interpolated positions. You must choose one as {interp} for the training stage:
* **'lin_approx'** : linear approximation.
* **'phys_model'** : physical model based on acceleration profile.

In [ ]:
# Choose Z-position interpolation method:
interp = 'lin_approx' # 'phys_model' or 'lin_approx'

# Prepare files information:
data_path =  'Data/Final_t_BxByBz_zAut_LabFrame/' # Datafiles path
gen_pars_path = 'ML_parameters/'

# Load data, general hyperparameters and rotational frames:
data, hypers, RFs = ML_MLmodel.load_data_and_gen_pars(
    data_path,gen_pars_path,interp=interp,final_stage=False)

## Generate time windows

### Load original data

Each data segment is processed into time windows with a fixed time length (or equivalently, fixed number of points). The distribution of training and testing datasets is chosen here. The validation dataset is included within the training dataset.

In [ ]:
# Prepare time windows:
wdw_pp = 20
train_segm = [0,2,4] if interp == 'phys_model' else [2,3,4]
t_wdws_train, t_wdws_test = ML_twdw.prepare_time_windows(
    data,wdw_pp,train_segm=train_segm,
    plot_instances=True,instances=10,start_wdw=1025,stride_pp=30)

### Machine Learning training

In this stage, each **[input magnetic components]** option will be combined with two architecture options:

1. A single Convolutional Neural Network (CNN) with 32 filters and kernel size 5 + Flattening layer + Two consecutive Dense Neural Network (DNN) with 512 neurons each.
2. Three consecutive DNN with 1024, 512, 512 neurons, respectively.

In both cases, the last output layer is a DNN with a single neuron and linear activation, which predicts a single value for the elevator Z-position, units [m], representing the position at the end of the time window.

The general hyperparameters for all ML models are defined according to the 'ML_gen_pars_S1to6.json' file. In addition, the following hyperparameters are also common to all ML models:

* "Time_Window_pp": {wdw_pp},
* "Dropout_Fraction": 0.
* "Activation_Function": 'relu'.
* "Optimizer": 'adam'.
* "Learning_Rate": 0.005.

In this stage, the standard rotational frames (RF1,RF2,RF3), described in the 'ML_RF_S1to6.json' file, are used to train and evaluate each ML model.

In [ ]:
# Define additional hyperparameters:
extra_hypers = {
    "Time_Window_pp": wdw_pp,
    "Dropout_Fraction": 0,
    "Activation_Function": 'relu',
    "Optimizer": 'adam',
    "Learning_Rate": 0.005
}

# Define the two architecture options:
arch1 = {
    "Convolutional_Network": True,
    "Conv_Layers": [[32,5]],
    "Pool_Layers": [None],
    "Flatten_Average": True,
    "Dens_Layers": [512,512],
    "Model_Name": "S1_C5_D512_D512"
}
arch2 = {
    "Convolutional_Network": False,
    "Dens_Layers": [1024,512,512],
    "Model_Name": "S1_D1024_D512_D512"
}

# Combine all hyperparameters information and architecture options:
arch_opts = [dict(list(hypers.items())+list(extra_hypers.items())+list(arch.items()))\
             for arch in [arch1,arch2]]

# Prepare rotational frame options:
RF_opts = list(RFs.values())

Define input magnetic component options:

In [ ]:
# Options for magnetic components:
B_comps_opts = [
    ['Bx'],['By'],['Bz'],['B'], # One component (scalar option included)
    ['Bx','By'],['Bx','Bz'], ['By','Bz'], # Two components
    ['Bx','By','Bz'] # Three components (full vector magnetometer)
    ]

Train and evaluate each architecture option in each available rotational frame for this stage and save the results in a .csv file (but not the models): 

In [ ]:
# Prepare file path to export results:
results_path = f'Results_{interp}/Train_s1s4s5_Test_s2s3/'
# Prepare file path to check on already trained models and avoid repetitions:
check_rep_model = f'Results_{interp}/Train_s1s4s5_Test_s2s3/Stage1_{interp}_all_Train_s1s4s5_Test_s2s3.csv'

# Train all models:
df_results = ML_MLmodel.train_stage1(
    B_comps_opts,arch_opts,RF_opts,t_wdws_train,t_wdws_test,
    results_path=results_path,check_rep_model=check_rep_model,
    quick_timing_test=False
)